In [1]:
import zipfile
import os

with zipfile.ZipFile('fruit-recognition.zip', 'r') as zip_ref:
    zip_ref.extractall('fruit_images')  # Wypakuj do określonego folderu

banany = []
gruszki = []
pomarancze = []

for root, dirs, files in os.walk('fruit_images'):
    for file in files:
        if 'Banana' in file:
            banany.append(os.path.join(root, file))
        elif 'Pear' in file:
            gruszki.append(os.path.join(root, file))
        elif 'Orange' in file:
            pomarancze.append(os.path.join(root, file))

print("Banany:", banany)
print("Gruszki:", gruszki)
print("Pomarańcze:", pomarancze)



Banany: ['fruit_images\\Banana\\71Banana02034.png', 'fruit_images\\Banana\\79Banana02063.png', 'fruit_images\\Banana\\87Banana02109.png', 'fruit_images\\Banana\\Banana01.png', 'fruit_images\\Banana\\Banana010.png', 'fruit_images\\Banana\\Banana0100.png', 'fruit_images\\Banana\\Banana01000.png', 'fruit_images\\Banana\\Banana01001.png', 'fruit_images\\Banana\\Banana01002.png', 'fruit_images\\Banana\\Banana01003.png', 'fruit_images\\Banana\\Banana01004.png', 'fruit_images\\Banana\\Banana01005.png', 'fruit_images\\Banana\\Banana01006.png', 'fruit_images\\Banana\\Banana01007.png', 'fruit_images\\Banana\\Banana01008.png', 'fruit_images\\Banana\\Banana01009.png', 'fruit_images\\Banana\\Banana0101.png', 'fruit_images\\Banana\\Banana01010.png', 'fruit_images\\Banana\\Banana01011.png', 'fruit_images\\Banana\\Banana01012.png', 'fruit_images\\Banana\\Banana01013.png', 'fruit_images\\Banana\\Banana01014.png', 'fruit_images\\Banana\\Banana01015.png', 'fruit_images\\Banana\\Banana01016.png', 'fruit_i

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import tkinter as tk
from PIL import Image, ImageTk
import cv2
import numpy as np
from tkinter import filedialog 

banany_path = "fruit_images\\Banana\\"
pomarancze_path = "fruit_images\\Orange\\"
gruszki_path = "fruit_images\\Pear\\"

input_shape = (224, 224, 3)
batch_size = 32
epochs = 10

X = []
y = []


for file in os.listdir(banany_path):
    img = load_img(os.path.join(banany_path, file), target_size=input_shape[:2])
    img = img_to_array(img) / 255.0
    X.append(img)
    y.append(0)  


for file in os.listdir(pomarancze_path):
    img = load_img(os.path.join(pomarancze_path, file), target_size=input_shape[:2])
    img = img_to_array(img) / 255.0
    X.append(img)
    y.append(1)  


for file in os.listdir(gruszki_path):
    img = load_img(os.path.join(gruszki_path, file), target_size=input_shape[:2])
    img = img_to_array(img) / 255.0
    X.append(img)
    y.append(2) 

X = np.array(X)
y = np.array(y)




In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
num_classes = 3

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
num_classes = 3

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

In [ ]:
train_loss, train_accuracy = model.evaluate(X_train, y_train)
print("Train Loss:", train_loss)
print("Train Accuracy:", train_accuracy)

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [ ]:
##intefejs 2
## działa lepiej
def process_image(image):

    resized_image = image.resize((64, 64))

    processed_image = np.array(resized_image)

    processed_image = processed_image / 255.0

    processed_image = np.expand_dims(processed_image, axis=0)

    return processed_image


def classify_image(image):
    predictions = model.predict(image)
    
    predicted_class_index = tf.argmax(predictions, axis=1).numpy()[0]

    labels = ['Banan', 'Pomarańcza', 'Gruszka', 'Inny owoc']
    predicted_class_label = labels[predicted_class_index]
    
    threshold = 0.5
    max_probability = np.max(predictions)
    if max_probability < threshold:
        predicted_class_label = "Inny owoc"
    
    return predicted_class_label


def upload_image():
    file_path = tk.filedialog.askopenfilename()
    if file_path:
        image = Image.open(file_path)
        processed_image = process_image(image)
        predicted_class = classify_image(processed_image)
        display_result(image, predicted_class)

def capture_image():
    camera = cv2.VideoCapture(0)
    _, frame = camera.read()
    camera.release()
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    processed_image = process_image(image)
    predicted_class = classify_image(processed_image)
    display_result(image, predicted_class)
    
def display_result(image, predicted_class):
    result_window = tk.Toplevel()
    
    image_tk = ImageTk.PhotoImage(image)
    
    image_label = tk.Label(result_window, image=image_tk)
    image_label.pack()
    
    class_label = tk.Label(result_window, text="Przewidziana klasa: " + predicted_class)
    class_label.pack()
    
    def close_window():
        result_window.destroy()
    
    close_button = tk.Button(result_window, text="Zamknij", command=close_window)
    close_button.pack()
    
    image_label.image = image_tk



# Inicjalizacja interfejsu użytkownika
root = tk.Tk()

root.configure(bg="green")

image = Image.open("owoce.jpg")

width, height = image.size
new_width = 400
new_height = int(new_width * height / width)
image = image.resize((new_width, new_height))

image_tk = ImageTk.PhotoImage(image)

image_label = tk.Label(root, image=image_tk)
image_label.pack()

upload_button = tk.Button(root, text="Wybierz obraz", command=upload_image)
upload_button.pack()

capture_button = tk.Button(root, text="Zrób zdjęcie", command=capture_image)
capture_button.pack()

root.mainloop()
